In [1]:
import pandas as pd
import numpy as np
import tqdm
import re
import scipy.stats as st
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
dataset = pd.read_csv("letras_final.csv", parse_dates=["date"])
dataset.drop(['Unnamed: 0', 'Unnamed: 0.1' ], axis = 1, inplace = True)
dataset.isna().sum()

artist_name           0
title                 0
album                 0
featured_artists      0
lyrics                0
date                273
url                   0
dtype: int64

In [3]:
dataset["artist_name"].value_counts()

Phoenix RDC              68
Sam The Kid              57
Valete                   53
ProfJam                  48
Boss AC                  46
Bispo                    43
Jimmy P                  42
Xeg                      42
Regula                   40
Allen Halloween          37
Dealema                  37
LON3R JOHNY              36
Da Weasel                35
Dillaz                   34
NGA                      34
GROGNation               30
Piruka                   30
Alcool Club              29
Agir                     26
Wet Bed Gang             26
Slow J                   25
Conjunto Corona          25
Keso                     23
Mike El Nite             21
Plutonio                 21
Deejay Telio             19
Kappa Jotta              19
X-Tense                  17
Papillon                 16
Força Suprema            16
Sir Scratch (PRT)        16
Carlão (PacMan)          15
Cálculo                  14
Chico da Tina            14
Estraca                  13
Waze                

In [4]:
#drop artists with less than 10 songs
counts = dataset['artist_name'].value_counts()
dataset = dataset[~dataset['artist_name'].isin(counts[counts < 10].index)]

In [5]:
#Dropping Deejay Telio
dataset = dataset.loc[dataset["artist_name"] != "Deejay Telio"]

In [6]:
dataset['lyrics'] = dataset['lyrics'].fillna(value='')

def remove_punct(s):
    new_lyr = re.sub(r'[^\w\s]', '', s).lower()
    new_lyr = re.sub(r'[\n]', ' ', new_lyr).lower()
    return new_lyr

dataset.lyrics = dataset.lyrics.apply(remove_punct)

In [7]:
#lyrics length
dataset["lyrics_len"] = [len(x) for x in dataset["lyrics"]]

In [8]:
# remover stopwords e tokenizar

stopwords_pt = nltk.corpus.stopwords.words('portuguese')
stopwords_pt.append("porque")
dataset['no_stopwords'] = dataset.apply(lambda x : ' '.join([word for word in x['lyrics'].split(' ') if word not in stopwords_pt]), axis = 1)
dataset['tokens'] = dataset['no_stopwords'].apply(nltk.word_tokenize)

In [9]:
dicionario = pd.read_csv("dicionario_limpo.csv")
dicionario

,Unnamed: 0,term,description,level,synonym,scale
0,1,abafa-palhinha,&bullet; homossexual masculino passivo,calão carroceiro,arrombado; paneleiro; larilas; lelé; bicha; b...,3
1,4,abananado,"admirado, aturdido e sem palavras",coloquial,estupefacto; azuratado; aturdido; embasbacado...,1
2,11,à beça,em grande quantidade,coloquial,NaN,1
3,13,abichanado,&bullet; homossexual masculino passivo,calão,arrombado; paneleiro; larilas; lelé; bicha; b...,2
4,20,abono de família,"órgão sexual masculino, pénis, pênis",coloquial,NaN,1
...,...,...,...,...,...,...
715,6508,xarro,NaN,coloquial,cavalo; heroína; charro,1
716,6511,xibungo,&bullet; homossexual masculino passivo,calão carroceiro,arrombado; paneleiro; larilas; lelé; bicha; b...,3
717,6512,xilindró,cadeia,coloquial,pildra; choça; prisão; prisa; calabouço; masm...,1
718,6518,xulo,é um:,calão carroceiro,none,3


In [10]:
#dropping irrelevant terms
dicionario = dicionario.loc[dicionario["term"] != "vale"]
dicionario = dicionario.loc[dicionario["term"] != "pinto"]
dicionario = dicionario.loc[dicionario["term"] != "tipo"]

In [11]:
#finding matches for swear words

lyrics_sets = dataset["tokens"]
swear_words = list(dicionario["term"])
scores = pd.Series(dicionario.scale.values,index=dicionario.term).to_dict()
score_data = []
swear_words_by_song = []

for lyrics in lyrics_sets:
  total_swear_score = 0
  swear_words_in_lyrics = [token for token in lyrics if token in swear_words]

  for swear in swear_words_in_lyrics:
      total_swear_score += scores.get(swear)

  swear_words_by_song.append(swear_words_in_lyrics)
  score_data.append(total_swear_score)

dataset["swears"] = swear_words_by_song
dataset["score"] = score_data

In [12]:
mean_by_artist = dataset.groupby('artist_name')['score'].mean()
mean_by_artist

artist_name
9 Miller             17.666667
Agir                  1.384615
Alcool Club           6.034483
Allen Halloween      11.945946
Bispo                 8.767442
Boss AC               2.173913
Carlão (PacMan)       4.733333
Chico da Tina        12.357143
Classe Crua           5.400000
Conjunto Corona       5.080000
Cálculo               4.214286
Da Weasel             3.371429
Dealema               4.648649
Dillaz                6.647059
Estraca               3.615385
Força Suprema         7.062500
Fínix MG              6.333333
GROGNation            7.366667
Holly Hood            5.727273
Jimmy P               2.309524
Kappa Jotta           7.473684
Keso                  5.565217
LON3R JOHNY           2.472222
Mike El Nite          5.190476
NGA                   8.941176
Papillon              5.687500
Phoenix RDC           9.117647
Piruka                6.800000
Plutonio              3.238095
ProfJam               7.125000
Regula                8.025000
Sam The Kid           5.438

In [13]:
#filtering misogynistic terms
lady_parts = dicionario[dicionario['description'].str.contains('órgão sexual feminino', na=False)]
men_parts = dicionario[dicionario['description'].str.contains('órgão sexual masculino', na=False)]
women_rude = dicionario[dicionario['synonym'].str.contains('prostituta', na=False)]
oral_sex = dicionario[dicionario['description'].str.contains('sexo oral', na=False)]
anal_sex = dicionario[dicionario['description'].str.contains('sexo anal', na=False)]
semen = dicionario[dicionario['description'].str.contains('esperma', na=False)]
cabra = dicionario[dicionario['term'].str.contains('cabra', na=False)]

dicionario_misoginia = pd.concat([lady_parts, women_rude, men_parts, semen, oral_sex, cabra], ignore_index= True)
dicionario_misoginia

,Unnamed: 0,term,description,level,synonym,scale
0,525,boceta,órgão sexual feminino,calão carroceiro,cona; pipi; pito; pitaço; pirona; rata; vagin...,3
1,977,cona,órgão sexual feminino,calão muito carroceiro,pipi; pito; pitaço; pirona; rata; vagina; nin...,4
2,978,conaça,órgão sexual feminino,calão muito carroceiro,cona; pipi; pito; pitaço; pirona; rata; vagin...,4
3,1106,crica,órgão sexual feminino,calão muito carroceiro,cona; pipi; pito; pitaço; pirona; rata; vagin...,4
4,1957,entrefolhos,órgão sexual feminino,calão,cona; pipi; pito; pitaço; pirona; rata; vagin...,2
...,...,...,...,...,...,...
75,2586,fazer um bobó,&bullet; sexo oral masculino,calão muito carroceiro,fazer um broche; mamada; chupa-me a pichota; ...,4
76,2589,fazer um broche,&bullet; sexo oral masculino,calão estupidamente carroceiro,fazer um bobó; mamada; chupa-me a pichota; fa...,5
77,2631,felácio,&bullet; sexo oral masculino,calão,fazer um broche; fazer um bobó; mamada; chupa...,2
78,3441,mamada,&bullet; sexo oral masculino,calão muito carroceiro,fazer um broche; fazer um bobó; chupa-me a pi...,4


In [14]:
#dicionario[dicionario['description'].str.contains('insulto a árbitro', na=False)]

In [15]:
lyrics_sets = dataset["tokens"]
misogynistic_words = list(dicionario_misoginia["term"])

misogyny_score = pd.Series(dicionario_misoginia.scale.values,index=dicionario_misoginia.term).to_dict()
misogyny_score_data = []
misogynistic_words_by_song = []

for lyrics in lyrics_sets:
  total_misogynistic_score = 0
  misogynistic_words_in_lyrics = [token for token in lyrics if token in misogynistic_words]

  for swear in misogynistic_words_in_lyrics:
      total_misogynistic_score += misogyny_score.get(swear)

  misogynistic_words_by_song.append(misogynistic_words_in_lyrics)
  misogyny_score_data.append(total_misogynistic_score)

dataset["misogynistic_swears"] = misogynistic_words_by_song
dataset["misogynistic_score"] = misogyny_score_data

In [16]:
# correlating misogyny with swearing
dataset['score'].corr(dataset["misogynistic_score"])

0.6301641736378527

In [17]:
#describing swearing and misogyny scores
columns_of_interest = ['score', 'misogynistic_score']
two_columns_of_data = dataset[columns_of_interest]
two_columns_of_data.describe()

,score,misogynistic_score
count,1127.000000,1127.000000
mean,6.121562,1.233363
std,9.840935,4.454389
min,0.000000,0.000000
25%,0.000000,0.000000
50%,3.000000,0.000000
75%,8.000000,0.000000
max,127.000000,81.000000


In [18]:
#describing misogyny per artist

misogyny_score_chart = pd.DataFrame()

misogyny_score_chart["Calão (média)"] = dataset.groupby(["artist_name"])["score"].mean()
misogyny_score_chart["Calão (mediana)"] = dataset.groupby(["artist_name"])["score"].median()
misogyny_score_chart["Calão (máx)"] = dataset.groupby(["artist_name"])["score"].max()
misogyny_score_chart["Misoginia (média)"] = dataset.groupby(["artist_name"])["misogynistic_score"].mean()
misogyny_score_chart["Misoginia (total)"] = dataset.groupby(["artist_name"])["misogynistic_score"].sum()
misogyny_score_chart["Misoginia (máx)"] = dataset.groupby(["artist_name"])["misogynistic_score"].max()
misogyny_score_chart.reset_index(inplace=True)
misogyny_score_chart.rename(columns={'artist_name': 'Artista'}, inplace = True)

misogyny_score_chart = misogyny_score_chart.round(decimals=2)
misogyny_score_chart.to_csv("misogyny_score_chart.csv")
misogyny_score_chart

,Artista,Calão (média),Calão (mediana),Calão (máx),Misoginia (média),Misoginia (total),Misoginia (máx)
0,9 Miller,17.67,14.0,44,6.83,82,21
1,Agir,1.38,0.0,13,0.00,0,0
2,Alcool Club,6.03,3.0,30,2.10,61,20
3,Allen Halloween,11.95,5.0,88,3.03,112,22
4,Bispo,8.77,6.0,70,0.30,13,3
5,Boss AC,2.17,1.0,23,0.52,24,9
6,Carlão (PacMan),4.73,0.0,50,0.33,5,2
7,Chico da Tina,12.36,9.5,48,2.79,39,15
8,Classe Crua,5.40,3.5,18,2.60,26,12
9,Conjunto Corona,5.08,2.0,29,1.68,42,15


,artist_name,title,album,featured_artists,lyrics,date,url,lyrics_len,no_stopwords,tokens,swears,score,misogynistic_swears,misogynistic_score
0,Mike El Nite,Só badalhocas,Vaporetto Titano,['Da Chick'],mike el nite só badalhocas nesta festa como é...,2016-01-01,https://genius.com/Mike-el-nite-so-badalhocas-...,2735,mike el nite badalhocas nesta festa deixaram ...,"[mike, el, nite, badalhocas, nesta, festa, dei...","[puta, foder, vaca, cu, puta, rabo, putaria]",19,"[puta, vaca, puta]",8
1,Mike El Nite,T.U.G.A.,O Justiceiro,[],hey vive a vida como uma festa sob o vento da ...,2016-04-22,https://genius.com/Mike-el-nite-tuga-lyrics,2628,hey vive vida festa sob vento floresta adoro r...,"[hey, vive, vida, festa, sob, vento, floresta,...","[gato, tipo, mágico]",3,[],0
2,Mike El Nite,Santa Maria,O Justiceiro,[],tu és loucura és o meu prazer uma aventura que...,2016-04-22,https://genius.com/Mike-el-nite-santa-maria-ly...,3079,és loucura és prazer aventura querer quero ter...,"[és, loucura, és, prazer, aventura, querer, qu...","[cabra, oferecida, merda, tipo, tipo, tipo, tipo]",11,[oferecida],2
3,Mike El Nite,Funeral,---,['Nerve'],é foi um foi um bom ano verdade mas isso não c...,2017-01-13,https://genius.com/Mike-el-nite-funeral-lyrics,2850,bom ano verdade chega nope vida presta faço c...,"[bom, ano, verdade, chega, nope, vida, presta,...","[puto, puto, puta, tipo]",8,[puta],3
4,Mike El Nite,Caixa Negra,Inter-Missão,['J-K (Rap)'],chamavamme desajeitado por não pintar dentro d...,2018-12-07,https://genius.com/Mike-el-nite-caixa-negra-ly...,2789,chamavamme desajeitado pintar dentro margem qu...,"[chamavamme, desajeitado, pintar, dentro, marg...","[vale, tipo, merda, tipo, mano, tipo, tipo]",10,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142,Jimmy P,Na Na Na,Momento da Verdade,['Celso OPP'],eu não tou pronto para um revés sou apenas mai...,NaT,https://genius.com/Jimmy-p-na-na-na-lyrics,1491,tou pronto revés apenas jovem procura ter mund...,"[tou, pronto, revés, apenas, jovem, procura, t...",[],0,[],0
1143,Jimmy P,Fonte da Juventude,#1,[],nunca há de haver ninguém que me ensine quem e...,NaT,https://genius.com/Jimmy-p-fonte-da-juventude-...,1945,nunca haver ninguém ensine passei sofri demais...,"[nunca, haver, ninguém, ensine, passei, sofri,...","[vale, vale]",4,[],0
1144,Jimmy P,Um Brinde,FVMILY F1RST,[],hoje eu faço um brinde a nós de tudo o que a v...,2015-02-20,https://genius.com/Jimmy-p-um-brinde-lyrics,1232,hoje faço brinde tudo vida deu ontem primavera...,"[hoje, faço, brinde, tudo, vida, deu, ontem, p...",[puto],2,[],0
1145,Jimmy P,A Luz,FVMILY F1RST,[],aqui ninguém derruba o que deus abençoou não o...,2015-02-26,https://genius.com/Jimmy-p-a-luz-lyrics,2491,aqui ninguém derruba deus abençoou oh todos ve...,"[aqui, ninguém, derruba, deus, abençoou, oh, t...",[],0,[],0


In [19]:
#dataset.to_csv("dataset.csv")


#describing unique words by artist
artist_word_unique = pd.DataFrame()
artist_word_unique["what"] = dataset.groupby(["artist_name", "no_stopwords"])

split_data = [list(i) for i in artist_word_unique["what"]]
artist_word_unique["Artist"] = [i[0] for i in split_data]
artist_word_unique["Lyrics"] = [i[1] for i in artist_word_unique["Artist"]]
artist_word_unique["Artist"] = [i[0] for i in artist_word_unique["Artist"]]
artist_word_unique

,what,Artist,Lyrics
0,"((9 Miller, antes nunca nunca beefem sério faç...",9 Miller,antes nunca nunca beefem sério façam ser pesso...
1,"((9 Miller, aqui dá pa ver boy perco tempo toc...",9 Miller,aqui dá pa ver boy perco tempo toca guita filh...
2,"((9 Miller, coração espancado rusga goe venha ...",9 Miller,coração espancado rusga goe venha cuspa prato ...
3,"((9 Miller, cuspir importante malta cospe pouc...",9 Miller,cuspir importante malta cospe pouco rua cospem...
4,"((9 Miller, han agora anda tudo preocupado hip...",9 Miller,han agora anda tudo preocupado hiphop sinceram...
...,...,...,...
1122,"((Xeg, yeh egotripping man contame histórias b...",Xeg,yeh egotripping man contame histórias boy quê ...
1123,"((Xeg, yeh niggas dont chill yeh boy xeg is ch...",Xeg,yeh niggas dont chill yeh boy xeg is chilling ...
1124,"((Xeg, yo mariana andote mancar panorama dread...",Xeg,yo mariana andote mancar panorama dreads andas...
1125,"((Xeg, yo xeg remisturas vol1 bemvindos trabal...",Xeg,yo xeg remisturas vol1 bemvindos trabalho ediç...


In [20]:
vc = artist_word_unique.Artist.value_counts()
vc = vc.reset_index()
vc.rename(columns={'Artist':'Number of Songs', 'index': 'Artist'}, inplace=True)
vc

,Artist,Number of Songs
0,Phoenix RDC,68
1,Sam The Kid,57
2,Valete,53
3,ProfJam,48
4,Boss AC,46
5,Bispo,43
6,Jimmy P,42
7,Xeg,42
8,Regula,40
9,Allen Halloween,37


In [21]:
#describing unique words by artist

def myfunc(x):
    s = [element for element in x]
    return " ".join(s)

df_perfect = artist_word_unique.groupby("Artist").agg({"Lyrics": lambda x: myfunc(x)})

real_unique = df_perfect.Lyrics.apply(lambda x: len(set([i.strip() for i in x.split(" ") if i != ""])))
real_unique = real_unique.reset_index()
real_unique.rename(columns={'Lyrics':'Palavras únicas'}, inplace=True)
real_unique = real_unique.merge(vc, how="left", on="Artist")
real_unique["ratio_unique"] = real_unique["Palavras únicas"] / real_unique["Number of Songs"]
real_unique.rename(columns={'Artist': 'Artista', "Number of Songs": 'Nº de Canções'}, inplace = True)
final_score = real_unique.merge(misogyny_score_chart, how="left", on="Artista")
final_score["Pontuação Final de Misoginia/Nº palavras"] = final_score["Pontuação total de misoginia"] / final_score["Palavras únicas"]
final_score["Pontuação Final de Misoginia/Canção"] = final_score["Pontuação total de misoginia"] / final_score["Nº de Canções"]

KeyError: 'Pontuação total de misoginia'

In [104]:
final_score.sort_values(by="Pontuação Final de Misoginia/Canção", ascending = False)
final_score.to_csv("classificação_final.csv")

In [105]:
dataset

,artist_name,title,album,featured_artists,lyrics,date,url,lyrics_len,no_stopwords,tokens,swears,score,misogynistic_swears,misogynistic_score
0,Mike El Nite,Só badalhocas,Vaporetto Titano,['Da Chick'],mike el nite só badalhocas nesta festa como é...,2016-01-01,https://genius.com/Mike-el-nite-so-badalhocas-...,2735,mike el nite badalhocas nesta festa deixaram ...,"[mike, el, nite, badalhocas, nesta, festa, dei...","[puta, foder, vaca, cu, puta, rabo, putaria]",19,"[puta, vaca, puta]",8
1,Mike El Nite,T.U.G.A.,O Justiceiro,[],hey vive a vida como uma festa sob o vento da ...,2016-04-22,https://genius.com/Mike-el-nite-tuga-lyrics,2628,hey vive vida festa sob vento floresta adoro r...,"[hey, vive, vida, festa, sob, vento, floresta,...","[gato, tipo, mágico]",3,[],0
2,Mike El Nite,Santa Maria,O Justiceiro,[],tu és loucura és o meu prazer uma aventura que...,2016-04-22,https://genius.com/Mike-el-nite-santa-maria-ly...,3079,és loucura és prazer aventura querer quero ter...,"[és, loucura, és, prazer, aventura, querer, qu...","[cabra, oferecida, merda, tipo, tipo, tipo, tipo]",11,"[cabra, oferecida]",4
3,Mike El Nite,Funeral,---,['Nerve'],é foi um foi um bom ano verdade mas isso não c...,2017-01-13,https://genius.com/Mike-el-nite-funeral-lyrics,2850,bom ano verdade chega nope vida presta faço c...,"[bom, ano, verdade, chega, nope, vida, presta,...","[puto, puto, puta, tipo]",8,[puta],3
4,Mike El Nite,Caixa Negra,Inter-Missão,['J-K (Rap)'],chamavamme desajeitado por não pintar dentro d...,2018-12-07,https://genius.com/Mike-el-nite-caixa-negra-ly...,2789,chamavamme desajeitado pintar dentro margem qu...,"[chamavamme, desajeitado, pintar, dentro, marg...","[vale, tipo, merda, tipo, mano, tipo, tipo]",10,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142,Jimmy P,Na Na Na,Momento da Verdade,['Celso OPP'],eu não tou pronto para um revés sou apenas mai...,NaT,https://genius.com/Jimmy-p-na-na-na-lyrics,1491,tou pronto revés apenas jovem procura ter mund...,"[tou, pronto, revés, apenas, jovem, procura, t...",[],0,[],0
1143,Jimmy P,Fonte da Juventude,#1,[],nunca há de haver ninguém que me ensine quem e...,NaT,https://genius.com/Jimmy-p-fonte-da-juventude-...,1945,nunca haver ninguém ensine passei sofri demais...,"[nunca, haver, ninguém, ensine, passei, sofri,...","[vale, vale]",4,[],0
1144,Jimmy P,Um Brinde,FVMILY F1RST,[],hoje eu faço um brinde a nós de tudo o que a v...,2015-02-20,https://genius.com/Jimmy-p-um-brinde-lyrics,1232,hoje faço brinde tudo vida deu ontem primavera...,"[hoje, faço, brinde, tudo, vida, deu, ontem, p...",[puto],2,[],0
1145,Jimmy P,A Luz,FVMILY F1RST,[],aqui ninguém derruba o que deus abençoou não o...,2015-02-26,https://genius.com/Jimmy-p-a-luz-lyrics,2491,aqui ninguém derruba deus abençoou oh todos ve...,"[aqui, ninguém, derruba, deus, abençoou, oh, t...",[],0,[],0


In [106]:
#palavras misóginas mais comuns

misogynistic_words = list(dicionario_misoginia["term"])

def increase_word_count(dictionary, word):
    dictionary[word] = (dictionary.get(word) or 0) + 1

misogynistic_term_counts = {}

misogynistic_swears = dataset["misogynistic_swears"]

all_misogynistic_swears_list = [token for token_list in misogynistic_swears for token in token_list]

for word in all_misogynistic_swears_list:
    increase_word_count(misogynistic_term_counts, word)

misogynistic_term_counts = pd.DataFrame.from_dict(misogynistic_term_counts, orient="index")
#misogynistic_term_counts.to_csv("contagem_palavras_misoginas.csv")

In [22]:
# palavras em calão mais comuns

swear_words = list(dicionario["term"])

def increase_word_count(dictionary, word):
    dictionary[word] = (dictionary.get(word) or 0) + 1

swear_term_counts = {}

normal_swears = dataset["swears"]

all_swears_list = [token for token_list in normal_swears for token in token_list]

for word in all_swears_list:
    increase_word_count(swear_term_counts, word)

swear_term_counts = pd.DataFrame.from_dict(swear_term_counts, orient="index")
swear_general_count = swear_term_counts.reset_index()
miso_general_count = misogynistic_term_counts.reset_index()
swear_general_count = swear_general_count.merge(miso_general_count, how="outer", on="index")

swear_general_count.rename(columns={'index':'Palavra',"0_x": "Calão", "0_y": "Calão misógino" }, inplace=True)
swear_general_count.to_csv("contagem_calão_todo.csv")

NameError: name 'misogynistic_term_counts' is not defined

NameError: name 'swear_term_counts' is not defined